In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
HOPSWORKS_PROJECT_NAME = "taxidemand_predict"

In [3]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [4]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
File 2023-11 was already in local storage
File 2023-12 was already in local 

In [5]:
print(f'{len(rides)=:,}')

len(rides)=143,216,677


In [6]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

/home/talha-naeem/Documents/LLM Work/taxi_demand/src/data.py:195: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
/home/talha-naeem/Documents/LLM Work/taxi_demand/src/data.py:158: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(ts_data['pickup_hour'].min(),
100%|██████████| 265/265 [00:10<00:00, 26.50it/s]


In [7]:
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [8]:
import hopsworks

In [9]:
project = hopsworks.login(
    project="taxidemand_predict",
    api_key_value=HOPSWORKS_API_KEY 
)

2025-07-31 23:03:21,068 INFO: Initializing external client
2025-07-31 23:03:21,068 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-07-31 23:03:24,883 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242261


In [10]:
feature_store = project.get_feature_store()

In [11]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 3

In [12]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

In [13]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1242261/fs/1224781/fg/1498480


Uploading Dataframe: 100.00% |██████████| Rows 8121720/8121720 | Elapsed Time: 10:24 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242261/jobs/named/time_series_hourly_feature_group_3_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_3_offline_fg_materialization', 'SPARK'),
 None)